In [1]:
### Goals
# 1. Create simple teams 
# 2. Have simple/damage bots battle with teams
# 3. Understand action space
# 4. Get dummy net to take actions in battle

"""
Run prior to NB

cd pokemon-showdown
cp config/config-example.js config/config.js
node pokemon-showdown start --no-security
"""

'\nRun prior to NB\n\ncd pokemon-showdown\ncp config/config-example.js config/config.js\nnode pokemon-showdown start --no-security\n'

In [ ]:
import asyncio
import numpy as np
import random
import torch
import torch.nn as nn

NUM_HP_BINS = 4

In [3]:
from poke_env import Player, RandomPlayer
from simple_dqn import DQN

In [4]:
class MaxDamagePlayer(Player):
    def choose_move(self, battle):
        # Chooses a move with the highest base power when possible
        if battle.available_moves:
            # Iterating over available moves to find the one with the highest base power
            best_move = max(battle.available_moves, key=lambda move: move.base_power)
            # Creating an order for the selected move
            return self.create_order(best_move)
        else:
            # If no attacking move is available, perform a random switch
            # This involves choosing a random move, which could be a switch or another available action
            return self.choose_random_move(battle)

In [5]:
class DebugPlayer(RandomPlayer):
    def choose_move(self, battle):
        # Print turn number
        print(f"Turn: {battle.turn}")

        # Print available moves
        print(f"Available moves: {[move.id for move in battle.available_moves]}")

        # Check battle has started
        your_pokemon = battle.active_pokemon
        opponent_pokemon = battle.opponent_active_pokemon
 
        # Extract key battle info
        state = {
            #"turn": battle.turn,
            "your_hp": int(NUM_HP_BINS * battle.active_pokemon.current_hp_fraction) / NUM_HP_BINS if your_pokemon else None,
            "your_status": battle.active_pokemon.status if your_pokemon else None,
            "your_moves": [move.id for move in battle.available_moves] if your_pokemon else None,
            "opponent_hp": int(NUM_HP_BINS * battle.opponent_active_pokemon.current_hp_fraction) / NUM_HP_BINS if opponent_pokemon else None,
            "opponent_status": battle.opponent_active_pokemon.status if opponent_pokemon else None
        }

        # Get type effectiveness
        moves_dmg_multiplier = np.ones(4)
        if battle.opponent_active_pokemon:
            for i, move in enumerate(battle.available_moves):
                if move.type:
                    moves_dmg_multiplier[i] = battle.opponent_active_pokemon.damage_multiplier(move)
        print(moves_dmg_multiplier)

        # Print the state info
        # print(f"State: {state}")

        return super().choose_move(battle)  # Acts as a random player

In [6]:
class DummyRLPlayer(Player):
    def __init__(self, model, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.model = model  # Store DQN model
        
    def choose_move(self, battle):
        # Check battle has started and get hp's
        your_pokemon = battle.active_pokemon
        opponent_pokemon = battle.opponent_active_pokemon

        your_hp = int(NUM_HP_BINS * battle.active_pokemon.current_hp_fraction) / NUM_HP_BINS if your_pokemon else -1
        opponent_hp = int(NUM_HP_BINS * battle.opponent_active_pokemon.current_hp_fraction) / NUM_HP_BINS if opponent_pokemon else -1

        # Get available move powers
        # -1 indicates that the move does not have a base power
        # or is not available
        moves_base_power = -np.ones(4)
        moves_dmg_multiplier = np.ones(4)
        if battle.opponent_active_pokemon:
            for i, move in enumerate(battle.available_moves):
                moves_base_power[i] = (
                    move.base_power / 100
                )  # Simple rescaling to facilitate learning
                if move.type:
                    moves_dmg_multiplier[i] = battle.opponent_active_pokemon.damage_multiplier(move)

        # test if state has worked correctly
        state = np.concatenate(
            [
                moves_base_power,
                moves_dmg_multiplier,
                [your_hp, opponent_hp],
            ]
        )
        print(state)

        # Create state vector
        state = np.concatenate([moves_base_power, moves_dmg_multiplier, [your_hp, opponent_hp]])
        state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0)  # Convert to tensor

        # Get Q-values from DQN
        with torch.no_grad():
            q_values = self.model(state_tensor)

        # Pick best move
        best_move_idx = torch.argmax(q_values).item()
        available_moves = battle.available_moves

        if available_moves and best_move_idx < len(available_moves):
            return self.create_order(available_moves[best_move_idx])
        else:
            return self.choose_random_move(battle)  # Fallback if move index is invalid


    def compute_reward(self, battle) -> float:
        pass

In [7]:
# Notes:
# must have some EV's on mons
# would be nice to check if mon is valid without losing kernel

team_1 = """
Charizard @ Heavy-Duty Boots
Ability: Blaze
EVs: 252 SpA / 4 SpD / 252 Spe
Timid Nature
- Roost
- Flamethrower
- Hurricane
- Toxic
"""
team_2 = """
Venusaur @ Life Orb
Ability: Chlorophyll
EVs: 252 SpA / 4 SpD / 252 Spe
Modest Nature
- Growth
- Giga Drain
- Weather Ball
- Sludge Bomb
"""
team_char = """
Charmander @ Leftovers
Ability: Blaze
EVs: 252 SpA / 4 SpD / 252 Spe
Adamant Nature
- Air Cutter
- Belly Drum
- Cut
- Ember
"""
team_bulb = """
Bulbasaur @ Leftovers
Ability: Overgrow
EVs: 252 SpA / 4 SpD / 252 Spe
Adamant Nature
- Amnesia
- Energy Ball
- Outrage
- Charm
"""
team_squir = """
Squirtle @ Leftovers
Ability: Rain Dish
EVs: 252 SpA / 4 SpD / 252 Spe
Adamant Nature
- Aqua Jet
- Bite
- Celebrate
- Hail
"""

In [8]:
async def random_player_battle(team_1, team_2, n_battles):
    p1 = DebugPlayer(battle_format="gen8ou", team=team_1)
    p2 = RandomPlayer(battle_format="gen8ou", team=team_2)

    await p1.battle_against(p2, n_battles=n_battles)

    for battle_tag, battle in p1.battles.items():
        won_txt = "lost"
        if battle.won:
            won_txt = "won"
        print("player 1 played battle: ", battle_tag, " and ", won_txt)

In [9]:
async def maxdam_player_battle(team_1, team_2, n_battles):
    p3 = MaxDamagePlayer(battle_format="gen8ou", team=team_1)
    p4 = MaxDamagePlayer(battle_format="gen8ou", team=team_2)

    await p3.battle_against(p4, n_battles=n_battles)

    for battle_tag, battle in p3.battles.items():
        won_txt = "lost"
        if battle.won:
            won_txt = "won"
        print("player 3 played battle: ", battle_tag, " and ", won_txt)

In [ ]:
#p1 = DebugPlayer(battle_format="gen8ou", team=team_bulb)
model = DQN() 
p1 = DummyRLPlayer(model, battle_format="gen8ou", team=team_bulb)
p2 = RandomPlayer(battle_format="gen8ou", team=team_char)

await p1.battle_against(p2, n_battles=1)

for battle_tag, battle in p1.battles.items():
    won_txt = "lost"
    if battle.won:
        won_txt = "won"
    print("player 1 played battle: ", battle_tag, " and ", won_txt)

2025-02-20 17:05:33,078 - DummyRLPlayer 1 - ERROR - Unhandled exception raised while handling message:
>battle-gen8ou-159
|
|t:|1740096333
|start
|switch|p1a: Bulbasaur|Bulbasaur, M|231/231
|switch|p2a: Charmander|Charmander, F|100/100
|turn|1
Traceback (most recent call last):
  File "/Users/mats/Documents/VSCode/poke_learning/venv/lib/python3.9/site-packages/poke_env/ps_client/ps_client.py", line 136, in _handle_message
    await self._handle_battle_message(split_messages)  # type: ignore
  File "/Users/mats/Documents/VSCode/poke_learning/venv/lib/python3.9/site-packages/poke_env/player/player.py", line 361, in _handle_battle_message
    await self._handle_battle_request(battle)
  File "/Users/mats/Documents/VSCode/poke_learning/venv/lib/python3.9/site-packages/poke_env/player/player.py", line 385, in _handle_battle_request
    choice = self.choose_move(battle)
  File "/var/folders/_j/k_sdq_0j0sb0mhzb4c3ldlr00000gn/T/ipykernel_33807/394302777.py", line 39, in choose_move
    state_te

[0.  0.9 1.2 0.  1.  0.5 1.  0.5 1.  1. ]


In [ ]:
"""if __name__ == "__main__":
    t1 = team_bulb
    t2 = team_squir
    task = asyncio.create_task(random_player_battle(t1, t2, 1))
    await task"""

'if __name__ == "__main__":\n    t1 = team_bulb\n    t2 = team_squir\n    task = asyncio.create_task(random_player_battle(t1, t2, 1))\n    await task'